In [1]:
! pip install requests pandas gspread

In [1]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from tapdb import TapDBClient
from gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS
import asyncio

target_date = (date.today() - timedelta(days=1)).isoformat()
# target_date = '2024-06-28'
if isinstance(target_date, str):
    target_date = date.fromisoformat(target_date)

async def create_tapdb():
    return TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

async def create_sheet():
    return DateSheet('[RP_CN] Revenue Report', date=target_date.isoformat())

task_tapdb = asyncio.create_task(create_tapdb())
task_sheet = asyncio.create_task(create_sheet())

await task_tapdb
await task_sheet

tapdb = task_tapdb.result()
sheet = task_sheet.result()

In [2]:
# [RP_CN] Revenue Report >> 活躍設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'active', dim='os', device=True)

sheet.update_cells('活躍設備\n(tap db)', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('活躍設備\n(android)', int(data.loc['Android', 'DAU_DEVICE']))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,36096,3980,32116,30627,27199,23504,18548
iOS,iOS,4728,2671,2057,1760,139,105,34


In [4]:
# [RP_CN] Revenue Report >> 新增設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'source', dim='os', device=True)
sheet.update_cells('新增設備\nTotal\n(Tap DB)', int(data['newDevices'].sum()))
sheet.update_cells('AND新設備\n(Tap DB)', int(data.loc['Android', 'newDevices']))
data


,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
Android,Android,5340,100,1576.9,54,439.2,4472,3930,87.88,33.74,None,2.24,1.21,0.10,8.13
iOS,iOS,971,22,267.7,9,20.4,3092,2835,91.69,50.79,None,0.71,0.29,0.01,2.27


In [5]:
# [RP_CN] Revenue Report >> 轉化率
data = tapdb.get_data('RP CN (PRO)', 'source', dim='ver')
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率')
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
8.0.0.353,8.0.0.353,6311,122,1844.6,63,459.6,7216,6564,90.96,40.34,None,1.69,0.87,0.06,7.3
1.0.0.146,1.0.0.146,0,0,0.0,0,0.0,174,112,64.37,41.90,None,0.00,0.00,0.00,0.0
7.0.0.325,7.0.0.325,0,0,0.0,0,0.0,82,44,53.66,22.95,None,0.00,0.00,0.00,0.0
8.0.0.352,8.0.0.352,0,0,0.0,0,0.0,56,27,48.21,33.05,None,0.00,0.00,0.00,0.0
6.0.0.274,6.0.0.274,0,0,0.0,0,0.0,14,9,64.29,40.00,None,0.00,0.00,0.00,0.0
7.0.0.308,7.0.0.308,0,0,0.0,0,0.0,10,6,60.00,31.82,None,0.00,0.00,0.00,0.0
6.0.0.273,6.0.0.273,0,0,0.0,0,0.0,4,2,50.00,36.36,None,0.00,0.00,0.00,0.0
2.0.0.178,2.0.0.178,0,0,0.0,0,0.0,2,0,0.00,33.33,None,0.00,0.00,0.00,0.0
1.0.0.166,1.0.0.166,0,0,0.0,0,0.0,2,0,0.00,NaN,None,0.00,0.00,0.00,0.0


In [12]:
# [RP_CN] Revenue Report >> 設備留存
data = tapdb.get_data('RP CN (PRO)', 'retain', begin_date='2024-04-01', pf='and', type='both')
data.sort_index(inplace=True)
data['key'] = data['key'].astype(str)
data_drr = data.loc[:, data.columns.str.contains('DRR')]
data = pd.concat([data.loc[:, ['newDevices']], data_drr / 100], axis=1)

data.to_csv('~/Desktop/retain.csv')
data

,newDevices,DRR1,DRR2,DRR3,DRR4,DRR5,DRR6,DRR7,DRR8,DRR9,...,DRR26,DRR27,DRR28,DRR29,DRR30,DRR60,DRR90,DRR120,DRR150,DRR180
key,,,,,,,,,,,,,,,,,,,,,
2024-04-01,3392,0.3420,0.2503,0.2114,0.1816,0.1689,0.1518,0.1456,0.1344,0.1203,...,0.0604,0.0596,0.0566,0.0522,0.0545,0.0239,0.0165,NaN,NaN,NaN
2024-04-02,3796,0.3311,0.2323,0.1955,0.1752,0.1610,0.1454,0.1394,0.1262,0.1117,...,0.0537,0.0522,0.0548,0.0493,0.0469,0.0224,0.0124,NaN,NaN,NaN
2024-04-03,6546,0.3262,0.1975,0.1573,0.1257,0.1121,0.1059,0.0930,0.0866,0.0880,...,0.0409,0.0530,0.0527,0.0466,0.0393,0.0182,0.0118,NaN,NaN,NaN
2024-04-04,10555,0.2928,0.1751,0.1122,0.1022,0.0968,0.0853,0.0798,0.0794,0.0950,...,0.0534,0.0626,0.0474,0.0418,0.0394,0.0124,NaN,NaN,NaN,NaN
2024-04-05,8652,0.2851,0.1472,0.1291,0.1153,0.1066,0.0993,0.1053,0.1192,0.0987,...,0.0624,0.0561,0.0520,0.0474,0.0452,0.0170,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-27,4348,0.3473,0.2523,0.2058,0.1851,0.1585,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-28,4472,0.3535,0.2361,0.1979,0.1717,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-29,5276,0.3237,0.2030,0.1585,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# [RP_CN] Revenue Report >> 活躍設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'wau', device=True)

target_week = target_date - timedelta(days=target_date.weekday())
row = data.loc[target_week]

# sheet.update_cells('活躍設備\n(tap db)', int(row['DAU_DEVICE']))
row

key              2024-07-08
WAU_DEVICE           101218
WAU_DEVICENew         43293
WAU_DEVICE2           57925
WAU_DEVICE3           47524
WAU_DEVICE4           42890
WAU_DEVICE5           39458
Name: 2024-07-08, dtype: object